In [ ]:
# get the code
import sys
import os
import matplotlib.pyplot as plt
# use seaborn plotting defaults
import seaborn as sns; sns.set()
sys.path.append('../code')

# import package functions
from codeCNV.plot import plot_genomic, plot_snp2, plot_snp, plot_2d, plot_3d
# import package functions
from script_utils_CNV import get_CNVconfig, show_output


# HOME
home = '/Users/mahtin'
home = '/Users/martinscience'

# standard paths
static = os.path.join(home, "Dropbox/Icke/Work/static")
tooldata = os.path.join(home, "Dropbox/Icke/Work/somVar/tooldata")
testdata = os.path.join(home,"Dropbox/Icke/Work/somVar/testdata")
PON_path = os.path.join(static, "PON/HAEv7_hg38_NovaSeq")
 
cnvdata = os.path.join(tooldata, "myCNVdata")
output_path = os.path.join(cnvdata, "output")
plot_path = os.path.join(cnvdata, "plot")
fig_path = os.path.join(cnvdata, "figures")

In [ ]:
path_config = dict(
        mawk_path="../shell",
        cov_path=os.path.join(output_path, "pile2CNV"),   # path containing rawcov.gz files for this sample
        snp_path=os.path.join(output_path, "pile2CNV"),   # path containing snp files for this sample
        bed_file=os.path.join(static, "bed_files/SureSelect/hg38/SS_HAEv7_hg38_Padded.bed"),
        genome_split_path=os.path.join(static, "genome/gatk/hg38/split"),
        gc_split_path=os.path.join(static, "genome/gatk/hg38/split"),
        genmap_split_path=os.path.join(static, "annotation/genmap/hg38/split"),
        PON_path = PON_path,
    )
CNVconfig = get_CNVconfig(
    "../config/config_CNV.yaml", 
    local_config=path_config)
CNVconfig

## extract blocks of CNV for clustering
+ needs to be done before the fusing with SNP in order to make use of high definition diff
+ 

In [ ]:
def get_blocks(df, col, min_size=0):
    '''
    takes a column of binary containment to certain group and returns block numbers and respective block_sizes
    excludes blocks below a certain block size limit
    '''
    
    org_cols = list(df.columns)
    # find gaps where col value changes
    df.loc[:, ['gap']] = (df[col] != df.shift(1)[col]).astype(int)
    # set the col names
    blocksize = f'{col}block_size'
    block = f'{col}block'
    
    # enumerate the gaps for blockID where value is 1
    df.loc[:, [block]] = df['gap'].cumsum() * df[col]
    # group by blocks and count size
    blocks = df.groupby(block)['gap'].count().rename(blocksize)
    # merge block size into df
    df = df.merge(blocks, left_on=block, right_index=True)
    # remove miniscule blocks
    df.loc[df[blocksize] < min_size, block] = 0
    
    ### maybe adjust block labelling to be numerically ordered
    # should maybe done 
    df.loc[:, col]  = df[block]
    # cols = org_cols + [block, blocksize]
    return df[org_cols]


def get_CNV_blocks(df, data, config):
    '''
    finds blocks of CNV with center LLH below threshold
    then it reduces these blocks to the regions within the Diff-peaks.. 
    ..to only enter the most meaningful data into clustering

    do the same thing for covCenter with values above center threshold
    (definitely belonging to the center)
    '''

    # extract params from config
    # for covLLH --> lookup combine.cov.LLH_cutoff t=cov
    t = data.replace('LLH', "")
    params = config[t]

    LLH_params = params['LLH_cutoff']

    col = data + "sum"
    diff_col = data + "Diff"

    # get the covCNV

    # get boolint whether LLH falls below threshold
    # fillna(1) to exclude any missing coverages
    df.loc[:, f'{t}CNV'] = (df[col].fillna(1) < LLH_params['cnv']).astype(int)
    # get the covCNV blocks
    df = get_blocks(df, f'{t}CNV', min_size=LLH_params['min_block_size'])
    # reduce data to within Diff-peaks
    df.loc[:, f'{t}CNVcore'] = ((df[f'{t}CNV'] > 0) & (
        df[f'{t}LLHsumDiff'] < LLH_params['max_diff'])).astype(int)

    # here I could also expand the covCNV to the peak of the Diff for covCNVexp
    # get the window_size for core expanding
    window_size = params['rolling_data'][data]['sum']

    # get the covCenter
    df.loc[:, f'{t}Center'] = (df[col].fillna(
        0) > LLH_params['center']).astype(int)
    # get the covCenter blocks
    df = get_blocks(df, f'{t}Center', min_size=LLH_params['min_block_size'])
    # reduce data to within Diff-peaks
    df.loc[:, f'{t}Centercore'] = ((df[f'{t}Center'] > 0) & (
        df[f'{t}LLHsumDiff'] < LLH_params['max_diff'])).astype(int)
    # get the window_size for core expanding
    window_size = params['rolling_data'][data]['sum']
    return df

In [ ]:
block_cov_df = get_CNV_blocks(rolling_cov_df, 'covLLH', config)
df

block_snp_df = get_CNV_blocks(snp_df, 'snpLLH', config)

## 2d-plot snpVAF vs log2ratio and assign CNVstatus for individual blocks

###  get the sample
+ from the sample_df

In [ ]:
def get_sample(sample):
    s = sample.replace("_", "")
    sample_df = pd.read_excel(os.path.join(wes_path, "cluster_samples.xlsx")).set_index('sample')
    return sample_df.loc[s, "file"]

get_sample("06A")

In [ ]:
sample = "06A"
cluster_df = pd.read_csv(get_sample(sample), sep='\t')
cluster_df

### strategy
+ get the means and sigma from the center_df
+ adjust absVAF and log2ratio of CNV_df
+ plot
+ find most likely purity by llh2d_mask

In [ ]:
def center_data(cluster_df):
    '''
    get the center_df containing only points belonging to both Centercores
    '''
    
    cols = ['log2ratio', 'log2ratiomean', 'absVAF', 'absVAFmean']

    # get the center_df with points belonging to both Centercores
    center_df = cluster_df.query('covCentercore + snpCentercore == 2').copy()
    # get the mean and std for relevant columns
    center_params = center_df[cols + ['VAF']].agg(['mean', 'std']).T
    # get the cnv_df with at least one CNVcore per point
    cnv_df = cluster_df.query('snpCNVcore + covCNVcore > 0').copy()
    
    for col in cols:
        center_df.loc[:, col] = center_df[col] - center_params.loc[col, 'mean']
        cnv_df.loc[:, col] = cnv_df[col] - center_params.loc[col, 'mean']
        
    cnv_df.loc[:, 'VAF'] = cnv_df['VAF'] + 0.5 - center_params.loc['VAF', 'mean']
    center_df.loc[:, 'VAF'] = center_df['VAF'] + 0.5 - center_params.loc['VAF', 'mean']
    return cnv_df, center_df, center_params

In [ ]:
cnv_df, center_df, center_params = center_data(cluster_df)
center_params

### 2-dimensional llh

In [ ]:
def llh2d(dx, dy, mx=0, my=0, sx=0.5, sy=0.1):
    '''
    compute the density function for a given gaussian
    takes a pd.Series or np.array
    '''   
    # get the fixed term
    s = 2 * np.pi * sx * sy
    return np.exp((((dx - mx) / sx) **2 + ((dy - my) / sy) **2) / -2) / s

### get all the means depending on alpha
+ will be used for making gaussians


In [ ]:
# start with a simple dataFrame
import math

def get_gauss_mask(alpha, Nmax):
    '''
    returns the gauss params for the gauss mask
    '''
    
    alpha = min(1,alpha)
    df = pd.DataFrame()
    for n in range(int(Nmax)):
        N = n + 1 
        for i in range(math.ceil((N + 1) / 2)):
            string = "A" * (N - i) + "B" * i
            if string == 'A':
                string = 'LOH'
            absVAF = alpha * np.abs((2 * i) / N - 1)
            log2 = np.log2(2 + alpha * (N - 2)) - 1
            s = pd.Series({'type': string, 'absVAF': absVAF, 'log2ratio': log2})
            df = df.append(s, ignore_index=True)
    return df.query('(absVAF !=0 or log2ratio != 0) or type == "AB"')


Nmax = 4
alpha = 1
gauss_mask = get_gauss_mask(alpha, Nmax)
gauss_mask

+ ####  mask2VAF for converting to VAF gaussians

In [ ]:
def mask2VAF(mask_df):
    '''
    converts the gauss_mask for absVAF into mask for VAF
    '''
    
    df = mask_df.copy()
    df.loc[:, 'VAF-'] = 0.5 - df['absVAF'] / 2
    df.loc[:, 'VAF+'] = 0.5 + df['absVAF'] / 2
    return df.loc[:, ['VAF-','VAF+', 'log2ratio', 'type']]


def VAFmask(alpha, Nmax=6):
    return mask2VAF(get_gauss_mask(alpha, Nmax))

VAFmask(alpha, Nmax)

In [ ]:
plt.style.use('seaborn-white')

def plot_gaussian(df, xcol, ycol, 
                  df2=pd.DataFrame(), 
                  Nmax=0,   # maximal expected
                  alpha=1,
                  gauss_params=pd.DataFrame(), # the center_params containing the std
                  logmax=2.5, 
                  std_factor = 1,
                  rings=10, # number of rings for contour
                  figsize=(6, 6)):
    fig, ax = plt.subplots(figsize=figsize)
    _ = ax.scatter(df[xcol], df[ycol], s=.1)
    if len(df2.index):
        _ = ax.scatter(df2[xcol], df2[ycol], s=2.5, alpha=.5, color='red')
    _ = ax.set_xlabel(xcol, fontsize=15)
    _ = ax.set_ylabel(ycol, fontsize=15)
    _ = ax.set_title(f"alpha={alpha}", fontsize=20)
    _ = sns.despine(ax=ax, offset=0)
    _ = ax.spines['left'].set_position('zero')

    
    def get_lims(col):
        if 'log' in col:
            return (-1.5, logmax)
        if 'VAF' in col:
            return (-0.05, 1.05)
    _ = ax.set_xlim(get_lims(xcol))
    _ = ax.set_ylim(get_lims(ycol))
    
    # add gaussian mask
    if Nmax:
        # create the grid
        x = np.linspace(*get_lims(xcol), 500)
        y = np.linspace(*get_lims(ycol), 400)
        X, Y = np.meshgrid(x, y)
        Z = np.zeros_like(X)
        
        # get the std from the params
        sx, sy = gauss_params.loc[[xcol, ycol], 'std']
        print(sx, sy)
        sx *= std_factor
        sy *= std_factor
        gaussians = get_gauss_mask(alpha, Nmax)
        if xcol == 'VAF':
            gaussians = mask2VAF(gaussians)
            for _, row in gaussians.iterrows():
                mx1 = row['VAF-']
                mx2 = row['VAF+']
                my = row['log2ratio']
                Z += llh2d(X,Y, mx1, my, sx, sy) +   llh2d(X,Y, mx2, my, sx, sy)
                ax.text(mx1, my-0.2, row['type'], ha='center')
                # add the mirror cluster unless it is AB
                if not row['type'] == "AB":
                    ax.text(mx2, my-0.2, row['type'].replace('A', 'G').replace('B', 'A').replace('G', 'B'), ha='center')
        else:
            for _, row in gaussians.iterrows():
                mx = row['absVAF']
                my = row['log2ratio']
                Z += llh2d(X,Y, mx, my, sx, sy)
                ax.text(mx, my-0.2, row['type'], ha='center')
        _ = ax.contour(X,Y,Z, rings, colors='black', alpha=1,linewidths=.4)
    # set the y-spine
    _ = ax.axhline(y=0, color='k')
    _ = ax.axvline(x=0, color='k')
    _ = ax.axvline(x=1, color='k')
    if xcol == 'VAF':
        _ = ax.axvline(x=0.5, c='k',ls="--")

    return fig, ax

## test samples

In [ ]:
log2 = dict(
        title='log2ratio',
        plot_type='scatter',   # ['line', 'scatter']
        data='log2ratio',
        plot_args=dict(
            linewidth=0.3,
            color='black',
            s=2,
            alpha=1
        )
    )

log2mean = dict(
        title='rollinglog2ratio',
        plot_type='line',   # ['line', 'scatter']
        data='log2ratiomean',
        plot_args=dict(
            linewidth=2,
            color='yellow',
            alpha=1
        )
    )
vaf = dict(
        title='VAF',
        plot_type='scatter',   # ['line', 'scatter']
        data='VAF',
        plot_args=dict(
            linewidth=1,
            color='black',
            s=3,
            alpha=1
        ))

fig_params = dict(
    figsize=(48,12),
    colormap='coolwarm_r',
    color_chroms=True,
    ylim=(0,1),
    cov_offset=.1,  # how much log2ratio=0 is shifted above SNP-data
    cov_height=.5,
    label_size=13
)

In [ ]:
sample = "04A"
cluster_df = pd.read_csv(get_sample(sample), sep='\t')
chroms = ['chr5', 'chr7','chr8', 'chr11', 'chr17']
cnv_df, center_df, center_params = center_data(cluster_df)
fig, _, _, _ = plot_snp2(cluster_df, snp_plots=[vaf], cov_plots=[log2, log2mean], blocks=["snpCNVcore", "covCNVcore"], chroms='all', region='chr7', **fig_params)
# fig.savefig(os.path.join(f"/Users/martinscience/Desktop/CNVAML/new/{sample}_blocks.jpg"),pil_kwargs=dict(quality=90))

In [ ]:
cluster_df['covCNV'].unique()

In [ ]:
cnv_df, center_df, center_params = center_data(cluster_df)
xcol = 'VAF'
ycol = 'log2ratiomean'
fig, ax = plot_gaussian(center_df, df2=cnv_df, xcol=xcol, ycol=ycol, 
                        Nmax=4, 
                        rings=8, 
                        logmax=1.5,
                        alpha=.85, 
                        std_factor=1,
                        gauss_params=center_params
                       )
# fig.savefig(os.path.join(f"/Users/martinscience/Desktop/CNVAML/new/{sample}_cluster.jpg"),pil_kwargs=dict(quality=90))

## do the block assignment according to LLH
+ LLH2D computation has to be done for every possible gaussian
+ data can be computed for entire CNV_df
+ grouped sums can then be maximized
+ center params are needed for gaussians

In [ ]:
def get_means(row):
    '''
    row helper for extracting the means from the VAFmask after extraction of alpha and CNVstatus
    '''
    return VAFmask(float(row['a'])).set_index("type").loc[row['type']]


def get_all_means(df):
    '''
    retrieves the means for VAF and log2ratiomean from the CNVcall for visualization
    '''
    
    df[['a', 'type']] = df['CNVcall'].str.extract(r"([0-9.]+)-([ABLOH]+)")
    df[['VAF-', 'VAF+', 'log2ratio']] = df.apply(get_means, axis=1)
    return df.drop(['a', 'type'], axis=1)


def call_blocks(df, alpha=0.9, Nmax=6, center_params=pd.DataFrame()):
    '''
    takes a cluster_df and makes the CNV calls
    1. computes the 2D-LLH for every given alpha and possible CNVstatus (VAF-symmetric)
    2. assigns the CNV-status per group where the LLH-sum is maximal
    3. outputs the blocks with CNV-status and imputed means for visualization
    '''
    # get the std from center_params
    vaf_std = center_params.loc['VAF', 'std']
    log2_std = center_params.loc['log2ratiomean', 'std']
    
    # force list for alpha
    if not isinstance(alpha, list):
        alpha = [alpha] 
    
    # add a column for every CNV type and every alpha
    # and calculate the respective 
    for a in alpha:
        # cycle through the VAFmask (containing the means for the respective gaussians)
        mask = VAFmask(a, Nmax)
        for _, row in mask.iterrows():
            df[f"{a}-{row['type']}"] = llh2d(df['VAF'],df['log2ratiomean'], mx=row['VAF-'], my=row['log2ratio'], sx=vaf_std, sy=log2_std)
            df[f"{a}-{row['type']}"] += llh2d(df['VAF'],df['log2ratiomean'], mx=row['VAF+'], my=row['log2ratio'], sx=vaf_std, sy=log2_std)
    
    # reduce df to the sums of LLH per CNVtype
    # reduce to required columns and group by snpCNV
    cnv_df = df.loc[:, ['snpCNV'] + [f"{a}-{m}" for a in alpha for m in mask['type']]].groupby('snpCNV').sum()
    
    cnv_df['CNVcall'] = cnv_df.columns[np.argmax(cnv_df.values, axis=1)]
    
    # get the start and end coordinates for each group
    region_df = df.loc[:, ['Chr', 'Pos', 'snpCNV']].groupby(['snpCNV', 'Chr'])['Pos'].agg(["min", "max"]).rename({'min':'Start', 'max': 'End'}, axis=1).reset_index('Chr')
    
    # merge the region into the cnv_df
    block_df = cnv_df.merge(region_df, left_index=True, right_index=True).loc[:, ['Chr', 'Start', 'End', 'CNVcall']]
    
    # add the VAF and log2ratio means for 
    # sort the blocks using key callable for quick chrom sort
    block_df = get_all_means(block_df).sort_values(by=['Chr', 'Start'], key=lambda x: x.astype(str).str.replace("chr", "").str.replace("X", "23").astype(int))
    return block_df   # .loc[block_df['VAF-'] != 0.5, :]

In [ ]:
df = cnv_df.copy()
block_df = call_blocks(df, [0.85, 0.4], 6, center_params)
block_df

## look at different data and scale-fit

In [ ]:
sample = "02A"
cluster_df = pd.read_csv(get_sample(sample), sep='\t')

fig, _, _, _ = plot_snp2(cluster_df, snp_plots=[vaf], cov_plots=[log2, log2mean], blocks=["snpCNVcore", "covCNVcore"], chroms='all', region='', **fig_params)
fig.savefig(os.path.join(f"/Users/martinscience/Desktop/CNVAML/new/{sample}_blocks.jpg"),pil_kwargs=dict(quality=90))

In [ ]:
cluster_dff = cluster_df.copy()
alpha = [.55,.7,.9]
scale = .75
shift = -.3
cluster_dff['log2ratiomean'] = (cluster_dff['log2ratiomean'] + shift) * scale
cnv_df, center_df, center_params = center_data(cluster_dff)
cnv_df['log2ratiomean'] = (cnv_df['log2ratiomean'] + shift) * scale
for a in alpha:
    fig, ax = plot_gaussian(center_df, df2=cnv_df, xcol=xcol, ycol=ycol, 
                        Nmax=4, 
                        rings=8, 
                        logmax=1.5,
                        alpha=a, 
                        std_factor=.7,
                        gauss_params=center_params
                       )
    fig.savefig(os.path.join(f"/Users/martinscience/Desktop/CNVAML/new/{sample}_{a}cluster.jpg"),pil_kwargs=dict(quality=90))
block_df = call_blocks(cnv_df, alpha, 6, center_params)


In [ ]:
block_df

In [ ]:
block_df.to_csv(os.path.join(wes_path, get_sample(sample).replace(".cluster", ".CNVcall")), sep='\t')